<a href="https://colab.research.google.com/github/yohoobot/works/blob/main/DaZhongDP_no.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium pandas



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 28.0 MB/s eta 0:00:00


In [1]:
!pip install selenium beautifulsoup4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 22.8 MB/s eta 0:00:00


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import tempfile
import time
import re

# ---------- 参数区域 ----------
SHOP_IDS = [
    "l7zG6MgC03O4BJJj",
    "HaRHYP99Aq1ULsE5",
    "l7IE99qK22kceYZy"
]
KEYWORDS = ["音乐", "bgm", "背景音乐", "播放", "轻音乐", "旋律", "安静", "吵", "爵士", "氛围"]
PAGE_WAIT = 5
PAGE_INTERVAL = 6
MAX_PAGES = 5

# ✅ 你提供的 Cookie：直接贴入即可
example_cookie_list = [
   ]

# ---------- 初始化浏览器 ----------
options = Options()
# ✅ 设置独立临时配置目录，解决“user-data-dir 被占用”错误
temp_profile = tempfile.mkdtemp()
options.add_argument(f"--user-data-dir={temp_profile}")
options.add_argument("--start-maximized")  # 启动时最大化（更稳定）

driver = webdriver.Chrome(options=options)

# ---------- 注入 Cookie ----------
def inject_cookies(driver, cookies):
    driver.get("https://www.dianping.com")
    time.sleep(3)
    for cookie in cookies:
        try:
            driver.add_cookie(cookie)
        except:
            pass

# ---------- 提取关键词评论 ----------
def extract_music_comments(html):
    soup = BeautifulSoup(html, "html.parser")
    comments = []
    divs = soup.find_all("div", class_=re.compile("MainReviewItem.*"))
    for div in divs:
        text = div.get_text(strip=True)
        if any(k in text for k in KEYWORDS):
            comments.append(text)
    return comments

# ---------- 主爬虫函数 ----------
def crawl_shop_comments(shop_id):
    url = f"https://www.dianping.com/shop/{shop_id}/review_all"
    all_comments = []

    driver.get(url)
    time.sleep(PAGE_WAIT)

    for page in range(1, MAX_PAGES + 1):
        print(f"📄 第 {page} 页：")
        html = driver.page_source
        comments = extract_music_comments(html)
        for c in comments:
            print("🎵", c)
        all_comments.extend(comments)

        # 尝试点击“下一页”
        try:
            next_btn = driver.find_element(By.CLASS_NAME, "NextPage")
            if "disabled" in next_btn.get_attribute("class"):
                print("🚫 到达最后一页")
                break
            next_btn.click()
            time.sleep(PAGE_INTERVAL)
        except Exception as e:
            print("⚠️ 翻页失败或已结束：", e)
            break

    return all_comments

# ---------- 主程序入口 ----------
if __name__ == "__main__":
    inject_cookies(driver, example_cookie_list)
    print("✅ 登录完成，开始爬取评论...\n")

    for shop_id in SHOP_IDS:
        print(f"\n=== 🍜 店铺 {shop_id} ===\n")
        comments = crawl_shop_comments(shop_id)
        print(f"\n🎯 共获取与音乐相关评论：{len(comments)} 条")
        print("=" * 60)

    driver.quit()


SessionNotCreatedException: Message: session not created: probably user data directory is already in use, please specify a unique value for --user-data-dir argument, or don't use --user-data-dir
Stacktrace:
#0 0x577293498ffa <unknown>
#1 0x577292f57970 <unknown>
#2 0x577292f9142a <unknown>
#3 0x577292f8d18f <unknown>
#4 0x577292fddbd9 <unknown>
#5 0x577292fdd106 <unknown>
#6 0x577292fcf063 <unknown>
#7 0x577292f9b328 <unknown>
#8 0x577292f9c491 <unknown>
#9 0x57729346042b <unknown>
#10 0x5772934642ec <unknown>
#11 0x577293447a22 <unknown>
#12 0x577293464e64 <unknown>
#13 0x57729342bbef <unknown>
#14 0x577293487558 <unknown>
#15 0x577293487736 <unknown>
#16 0x577293497e76 <unknown>
#17 0x7a540bdb2ac3 <unknown>
